# DB 연결 및 데이터 테스트

이 노트북은 Flask 애플리케이션의 데이터베이스에 연결하고 `RealEstateTransaction` 및 `PublicParking` 모델에서 데이터를 가져옵니다.

In [141]:
from utils import setup_db_context
import pandas as pd
from sqlalchemy import select

# DB 컨텍스트 설정
app, db = setup_db_context()

from myapp.models import RealEstateTransaction, PublicParking



데이터베이스 연결 및 앱 컨텍스트 푸시 완료.


## 1. 부동산 실거래가 (Real Estate Transactions)

In [142]:
# SQLAlchemy를 사용하여 데이터 가져오기
transactions = RealEstateTransaction.query.limit(5).all()
print(f"가져온 거래 수: {len(transactions)}")

# DataFrame으로 변환
stmt = select(RealEstateTransaction)
df_ret = pd.read_sql(stmt, db.session.connection())

print(f"총 레코드 수: {len(df_ret)}")
df_ret.head()



가져온 거래 수: 5
총 레코드 수: 235671


,ret_id,reception_year,district_code,district_name,legal_dong_code,legal_dong_name,jibun_type,jibun_type_name,main_number,sub_number,...,amount,building_area,land_area,floor,right_type,cancel_date,construction_year,building_use,declaration_type,broker_district_name
0,2,2025,11320,도봉구,10600,방학동,1,대지,0638,0010,...,14800,37.66,16.0,4,,,1994.0,연립다세대,중개거래,서울 도봉구
1,3,2025,11380,은평구,10800,역촌동,,,,,...,39620,74.08,182.0,,,,1972.0,단독다가구,직거래,
2,4,2025,11500,강서구,10200,등촌동,1,대지,0628,0013,...,10300,26.16,35.71,6,,,1999.0,오피스텔,중개거래,서울 강서구
3,5,2025,11620,관악구,10200,신림동,1,대지,1639,0051,...,12700,18.70,26.16,8,,,2014.0,오피스텔,중개거래,서울 관악구
4,6,2025,11530,구로구,10600,고척동,1,대지,0339,0000,...,84000,59.89,0.0,9,,,2009.0,아파트,중개거래,"서울 구로구, 서울 양천구"


In [143]:
# 1. 사용자로부터 직접 입력 받기
try:
    target_budget = int(input("최대 예산을 입력하세요 (단위: 만원, 예: 10000): "))
    target_gu = input("조회할 '구' 이름을 입력하세요 (예: 강남구): ")
except ValueError:
    print("예산은 숫자만 입력 가능합니다. 기본값(10000)으로 설정합니다.")
    target_budget = 10000

# 2. 해당 구의 데이터 1차 필터링 및 동 리스트 추출
#df_gu = df_ret[df_ret["district_name"] == target_gu].copy()

while df_gu.empty:
    print(f"'{target_gu}'에 해당하는 데이터가 없습니다. 구 이름을 확인해 주세요.")
    target_gu = input("구 이름이 없습니다. 다시 입력해주세요 (예: 강남구): ").strip()
    df_gu = df_ret[df_ret["district_name"] == target_gu].copy()

df_gu = df_ret[df_ret["district_name"] == target_gu].copy()

if df_gu.empty:
    print(f"'{target_gu}'에 해당하는 데이터가 없습니다. 구 이름을 확인해 주세요.")
else:
    dong_list = df_gu["legal_dong_name"].unique()
    print(f"\n[{target_gu}] 내 선택 가능한 동: {list(dong_list)}")
    
    # 동 이름 입력 받기
    target_dong = input("조회할 '동' 이름을 입력하세요 (예: 대조동): ")

    # 3. 최종 필터링: 구 + 동 + 예산(amount) 조건 적용
    condition = (
        (df_ret["district_name"] == target_gu) & 
        (df_ret["legal_dong_name"] == target_dong) & 
        (df_ret["amount"] <= target_budget)
    )
    df_fields = df_ret[condition].copy()

    # 4. 결과 출력
    print(f"\n" + "="*30)
    print(f" 필터링 결과")
    print(f" - 선택 지역: {target_gu} {target_dong}")
    print(f" - 설정 예산: {target_budget:,}만원 이하")
    print(f" - 추출 데이터: {len(df_fields)}건")
    print("="*30)

    if not df_fields.empty:
        display(df_fields.head())
    else:
        print("조건에 맞는 매물이 없습니다. 예산을 높이거나 지역을 변경해 보세요.")



[강남구] 내 선택 가능한 동: ['도곡동', '역삼동', '수서동', '대치동', '논현동', '청담동', '삼성동', '압구정동', '개포동', '자곡동', '일원동', '세곡동', '율현동', '신사동']

 필터링 결과
 - 선택 지역: 강남구 대치동
 - 설정 예산: 80,000만원 이하
 - 추출 데이터: 391건


,ret_id,reception_year,district_code,district_name,legal_dong_code,legal_dong_name,jibun_type,jibun_type_name,main_number,sub_number,...,amount,building_area,land_area,floor,right_type,cancel_date,construction_year,building_use,declaration_type,broker_district_name
286,288,2025,11680,강남구,10600,대치동,1,대지,0889,0005,...,29800,35.70,35.7,16,,,1990.0,오피스텔,중개거래,서울 강남구
478,480,2025,11680,강남구,10600,대치동,1,대지,0890,0059,...,43000,44.46,62.09,19,,,2004.0,오피스텔,중개거래,서울 강남구
711,713,2025,11680,강남구,10600,대치동,1,대지,0925,0000,...,70000,51.50,29.0,2,,,2019.0,연립다세대,직거래,
4240,4242,2024,11680,강남구,10600,대치동,1,대지,0891,0023,...,25000,34.20,43.39,2,,,2004.0,오피스텔,중개거래,경기 성남시 수정구
4445,4447,2024,11680,강남구,10600,대치동,1,대지,0890,0059,...,29500,39.16,54.68,8,,,2004.0,오피스텔,중개거래,서울 강남구
